In [1]:
import os
import random

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [5, 5]
matplotlib.rcParams['figure.dpi'] = 200

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

from data_helper import UnlabeledDataset, LabeledDataset
from helper import collate_fn, draw_box

In [2]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0);

In [3]:
# All the images are saved in image_folder
# All the labels are saved in the annotation_csv file
image_folder = '../data'
annotation_csv = '../data/annotation.csv'

In [4]:
labeled_scene_index = np.arange(106, 134)
transform = torchvision.transforms.ToTensor()

# The labeled dataset can only be retrieved by sample.
# And all the returned data are tuple of tensors, since bounding boxes may have different size
# You can choose whether the loader returns the extra_info. It is optional. You don't have to use it.
labeled_trainset = LabeledDataset(image_folder=image_folder,
                                  annotation_file=annotation_csv,
                                  scene_index=labeled_scene_index,
                                  transform=transform,
                                  extra_info=False
                                 )
trainloader = torch.utils.data.DataLoader(labeled_trainset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)

In [6]:
sample, target, road_image = iter(trainloader).next()
print(torch.stack(sample).shape)

torch.Size([2, 6, 3, 256, 306])


In [7]:
from bbox_helperfunc import *

In [9]:
print(len(target))

2


In [10]:
label = bbox_to_label(target)

In [13]:
print(label.shape)
print(label)
print(label.sum())

(800, 800)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
4649.0


In [14]:
test_bbox = get_bboxes_from_output(label)

In [15]:
print(test_bbox.shape)
print(test_bbox)

torch.Size([5, 2, 4])
tensor([[[ 18.2000,  18.2000,  22.9000,  22.9000],
         [ -4.3000,  -2.4000,  -4.3000,  -2.4000]],

        [[-23.6000, -23.6000, -19.0000, -19.0000],
         [ -0.9000,   1.0000,  -0.9000,   1.0000]],

        [[  7.4000,   7.4000,  12.0000,  12.0000],
         [  2.7000,   4.7000,   2.7000,   4.7000]],

        [[ 34.0000,  34.0000,  39.0000,  39.0000],
         [  5.8000,   7.9000,   5.8000,   7.9000]],

        [[-12.8000, -12.8000,  -8.0000,  -8.0000],
         [  6.3000,   8.2000,   6.3000,   8.2000]]], dtype=torch.float64)


In [21]:
new = []
new.append(test_bbox)
new.append(target[1]['bounding_box'])
new

[tensor([[[ 18.2000,  18.2000,  22.9000,  22.9000],
          [ -4.3000,  -2.4000,  -4.3000,  -2.4000]],
 
         [[-23.6000, -23.6000, -19.0000, -19.0000],
          [ -0.9000,   1.0000,  -0.9000,   1.0000]],
 
         [[  7.4000,   7.4000,  12.0000,  12.0000],
          [  2.7000,   4.7000,   2.7000,   4.7000]],
 
         [[ 34.0000,  34.0000,  39.0000,  39.0000],
          [  5.8000,   7.9000,   5.8000,   7.9000]],
 
         [[-12.8000, -12.8000,  -8.0000,  -8.0000],
          [  6.3000,   8.2000,   6.3000,   8.2000]]], dtype=torch.float64),
 tensor([[[-10.6664, -10.6752, -16.2207, -16.2296],
          [ -2.2139,  -4.3386,  -2.1922,  -4.3169]],
 
         [[ -3.7310,  -3.7401,  -9.1664,  -9.1754],
          [ -2.1635,  -4.3332,  -2.1422,  -4.3119]],
 
         [[  7.2263,   7.2182,   2.3299,   2.3218],
          [ -2.4688,  -4.4135,  -2.4497,  -4.3944]],
 
         [[ 15.6763,  15.6526,  11.3561,  11.3324],
          [  4.4237,   2.5441,   4.4771,   2.5975]],
 
         [[ 23.9

In [16]:
print(target[0]['bounding_box'])

tensor([[[ 39.0656,  39.0420,  34.0411,  34.0175],
         [  7.9216,   5.8571,   7.9782,   5.9137]],

        [[-18.9801, -18.9814, -23.5589, -23.5603],
         [  1.0340,  -0.8957,   1.0364,  -0.8933]],

        [[ 22.9782,  22.9435,  18.3121,  18.2774],
         [ -2.3560,  -4.2673,  -2.2720,  -4.1834]],

        [[ 12.0548,  12.0257,   7.4764,   7.4473],
         [  4.7168,   2.7304,   4.7832,   2.7967]],

        [[ -7.9940,  -7.9953, -12.7729, -12.7741],
         [  8.2002,   6.3935,   8.2027,   6.3960]]], dtype=torch.float64)


In [17]:
from helper import compute_ats_bounding_boxes

compute_ats_bounding_boxes(test_bbox, target[0]['bounding_box'])

tensor(0.9503)